<a href="https://colab.research.google.com/github/VioletKarasik/Colaboratory/blob/main/para2task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import requests
import json
import pandas as pd
from datetime import datetime
import time

# API ключ из config.json
with open('config.json', 'r') as f:
    config = json.load(f)

# Параметры API запроса
symbols = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT']  # Символы криптовалют
interval = '1d'  # Интервал данных - 1 день
start_date = '2014-01-01'  # Начальная дата
end_date = datetime.now().strftime('%Y-%m-%d')  # Текущая дата

def date_to_millis(date_str):
    dt = datetime.strptime(date_str, '%Y-%m-%d')
    return int(dt.timestamp() * 1000)

# Получение данных
def get_historical_data(symbol, interval, start_date, end_date):
    url = 'https://api.binance.com/api/v3/klines'
    start_time = date_to_millis(start_date)
    end_time = date_to_millis(end_date)
    data = []

    while start_time < end_time:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'limit': 1000
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Ошибка запроса данных для {symbol}!")
            break

        klines = response.json()
        if not klines:
            break

        # Данные и время
        data.extend(klines)
        start_time = klines[-1][0] + 1  # Следующий запрос со следующего дня
        time.sleep(0.1)  # Задержка для избежания лимита

    return data

# Общий DataFrame для хранения данных всех монет
final_data = []

# Сохранение данных и расчёт доходности для каждой криптовалюты
for symbol in symbols:
    data = get_historical_data(symbol, interval, start_date, end_date)
    columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time',
               'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume',
               'Taker Buy Quote Asset Volume', 'Ignore']
    df = pd.DataFrame(data, columns=columns)
    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
    df['Close'] = df['Close'].astype(float)

    # Годовая доходность
    df['Year'] = df['Open Time'].dt.year
    yearly_returns = df.groupby('Year')['Close'].apply(lambda x: (x.iloc[-1] - x.iloc[0]) / x.iloc[0] * 100)

    # Среднегодовая доходность
    avg_annual_return = yearly_returns.mean()

    # Данные в финальный список
    for year, return_rate in yearly_returns.items():
        final_data.append({
            'Symbol': symbol,
            'Year': year,
            'Yearly Return (%)': return_rate,
            'Average Annual Return (%)': avg_annual_return if year == yearly_returns.index[-1] else None
        })

# Итоговые данные  в CSV
final_df = pd.DataFrame(final_data)
final_df.to_csv('crypto_returns_summary.csv', index=False)
print('Все данные сохранены в crypto_returns_summary.csv')

Все данные сохранены в crypto_returns_summary.csv
